# Library Management system

In [1]:
from mysql import connector # API for MySQL
import pandas as pd # pandas' .read_sql_query() will be used to read the MySQl query in python
import hashlib # passwods in username are stored in hashes not strings. In order to change the passwords user_input string to hash 
from getpass import getpass # used instead of input() which will not reveal the typed password unlike input()
from user import username # imports dictionary containging usernames and passwords
import auth # contains username and password information for MySQL

In [2]:
con = connector.connect(user=auth.user,password=auth.password,host=auth.host,database=auth.database)
my_cursor = con.cursor()

## Creating tables

In [3]:
# creating table books 
# my_cursor.execute('''
# create table books(
# b_id int primary key,
# b_name varchar(50),
# b_author varchar(25),
# b_genre varchar(25),
# b_availability varchar(25)
# );
# ''')
# con.commit()

In [4]:
# creating table book_issue 
# my_cursor.execute('''
# create table book_issue(
# r_id int references reader(r_id),
# b_id int references books(b_id)
# );
# ''')
# con.commit()

In [5]:
# creating table reader 
# my_cursor.execute('''
# create table reader(
# r_id int primary key,
# r_name varchar(25)
# );
# ''')
# con.commit()

## Inserting values to the tables

In [6]:
# inserting books values
# my_cursor.execute('''
# insert into books values
# (1, 'Harry Potter and the Deathly Hallows', 'J. K. Rowling', 'Fantasy', 'Available'),
# (2, 'Harry Potter and the Prisoner of Azkaban', 'J. K. Rowling', 'Fantasy', 'Issued'),
# (3, 'Harry Potter and the Philosophers Stone', 'J. K. Rowling', 'Fantasy', 'Available'),
# (4, 'Harry Potter and the Goblet of Fire', 'J. K. Rowling', 'Fantasy', 'Available'),
# (5, 'Blood of Elves', 'Andrzej Sapkowski', 'Fantasy', 'Issued'),
# (6, 'Time of Contempt ', 'Andrzej Sapkowski', 'Fantasy', 'Available'),
# (7, 'Baptism of Fire ', 'Andrzej Sapkowski', 'Fantasy', 'Available'),
# (8, 'The Tower of the Swallow', 'Andrzej Sapkowski', 'Fantasy', 'Issued'),
# (9, 'The Lady of the Lake', 'Andrzej Sapkowski', 'Fantasy', 'Issued');
# ''')
# con.commit()

In [7]:
# inserting reader values
# my_cursor.execute('''
# insert into reader values
# (1001,'Arun'),
# (1002,'Rick'),
# (1003,'Morty'),
# (1004,'Summer');
# ''')
# con.commit()

In [8]:
# inserting book_issue values
# my_cursor.execute('''
# insert into book_issue values
# (1001,4),
# (1002,8),
# (1003,2),
# (1004,9);
# ''')
# con.commit()

## Viewing records of the tables

In [9]:
def table_books():
    """Views table books."""
    print('Table : books')
    display(pd.read_sql_query('select * from books', con))

In [10]:
def table_reader():
    """Views table reader."""
    print('Table : reader')
    display(pd.read_sql_query('select * from reader', con))

In [11]:
def table_book_issue():
    """Views table book_issue."""
    print('Table : book_issue')
    display(pd.read_sql_query('select * from book_issue', con))

## Issue and Return book


In [12]:
def issue_book():
    """issue book function  create the record to the table book_issue
    and updates books table."""
    table_reader()
    table_books()

    user_id = int(input('Enter user ID : '))

    if user_id in pd.read_sql_query('select * from reader', con)['r_id'].to_list():
        book_id = input('Enter the book id : ')
        if int(book_id) not in \
                pd.read_sql_query('select b_id from book_issue', con)['b_id'].to_list():
            my_cursor.execute(f'INSERT INTO book_issue VALUES({user_id},{book_id});')
            my_cursor.execute(f"update books set b_availability = 'Issued' where b_id = {book_id};")
            con.commit()
            print('Book Successfully updated.\n')
        else:
            print('Book is not available for issue.\n')
    else:
        print('User not found!\n')

In [13]:
def return_book():
    """return book function returns the delete the record from the table book_issue
    and updates books table."""
    table_book_issue()

    user_id = int(input('Enter user ID : '))
    if user_id not in pd.read_sql_query('select * from book_issue', con)['r_id'].to_list():
        print('User ID is wrong.\n')
    else:
        book_id = int(input('Enter the book id : '))
        if book_id not in pd.read_sql_query('select * from book_issue', con)['b_id'].to_list():
            print('Book ID is wrong.\n')

        else:
            my_cursor.execute(f'DELETE from book_issue where b_id ={book_id} and r_id={user_id}')
            my_cursor.execute(f'update books set b_availability = \'Available\' where b_id ={book_id}')
            con.commit()
            print('Book Successfully returned.\n')

## Upload records in the table

In [14]:
def add_new_reader():
    """add new book reader adds new reader record to the tables reader."""
    table_reader()
    
    reader_id, reader_name = int(pd.read_sql_query('''
        select r_id from reader order by r_id desc limit 1
        ''', con).values[0][0]) + 1, input(
        'Enter name of the reader : ')
    try:
        my_cursor.execute(f'insert into reader values({reader_id},\'{reader_name}\');')
        con.commit()
        print(f'Successfully added the book {reader_name} to table reader.\n')
    except connector.Error as error:
        print('Update failed!\n')
        print(f'Error : {error}')

In [15]:
def add_new_book():
    """add new book function adds new book record to the tables book."""
    table_books()
    
    b_id = int(pd.read_sql_query('''
    select b_id from books order by b_id desc limit 1
    ''', con).values[0][0]) + 1
    b_name = input('Enter the name of the book : ')
    b_author = input('Enter the name of the Author : ')
    b_genre = input('Enter the name of the Genre : ')
    
    try:
        my_cursor.execute(f"insert into books values({b_id}, '{b_name}', '{b_author}', '{b_genre}', 'Available')")
        con.commit()
        print(f'Successfully added {b_name} user to table reader.\n')
    except connector.Error as error:
        print(f'Error : {error}')
        print('Update failed!\n')

## Modify records in the tables

In [16]:
def modify_reader():
    """modify reader function modifies the reader record from the tables reader."""
    table_reader()
    
    modify_readers = int(input('enter the reader id'))
    
    if modify_readers not in pd.read_sql_query('select * from reader', con)['r_id'].to_list():
        print('User ID is wrong.\n')
    else:
        r_name = input('Enter new name of the reader')
        my_cursor.execute(f"update reader set r_name = '{r_name}'where r_id = {modify_readers};")
        con.commit()
        print('Reader Successfully modified.\n')

In [17]:
def modify_book():
    """modify book function modifies the book record from the tables book."""
    table_books()
    
    modify_books = int(input('enter the book id'))
    
    if modify_books not in pd.read_sql_query('select * from books', con)['b_id'].to_list():
        print('Book ID is wrong.\n')
    else:
        b_name, b_author, b_genre = input(
            'Enter new name of the book'), input(
            'Enter new Author name of the book'), input(
            'Enter new Genre of the book')
        my_cursor.execute(
            f'''
            update books 
            set b_name = '{b_name}', 
            b_author = '{b_author}', 
            b_genre = '{b_genre}' 
            where b_id = {modify_books};
            ''')
        con.commit()
        print('Book Successfully modified.\n')

## Remove records from tables

In [18]:
def remove_book():
    """remove book function removes the book record from the tables book."""
    table_books()
    
    book_id = int(input('Enter book ID : '))

    if book_id not in pd.read_sql_query('select * from books', con)['b_id'].to_list():
        print('Book ID is wrong.\n')
    else:
        query = pd.read_sql_query(f'select * from books where b_id = {book_id}', con)
        if (query['b_availability'] == 'Issued')[0]:
            print('Issued book cannot be removed.\n')
        else:
            my_cursor.execute(f'delete from books where b_id = {book_id}')
            con.commit()
            print('Book Successfully Deleted.\n')

In [19]:
def remove_reader():
    """remove reader function removes the reader record from the tables reader."""
    table_reader()
    
    remove_user = int(input('enter the user id'))
    
    if remove_user not in pd.read_sql_query('select * from reader', con)['r_id'].to_list():
        print('User ID is wrong.\n')
    elif remove_user in pd.read_sql_query('select r_id from book_issue', con)['r_id'].to_list():
        print('User cannot be removed before returning the book.\n')
    else:
        my_cursor.execute(f'delete from reader where r_id = {remove_user}')
        con.commit()
        print('User Successfully Deleted.\n')

## Main program

In [20]:
def select_table(fn1, fn2, fn3=None):
    """Select function use to get the tables names."""
    print([
              'Please select a table :\n1. Books\n2. Reader' \
                  if fn3 is None else 'Please select a table :\n1. Books\n2. Reader\n3. Book issue'][0])
    user_input_remove = input()
    if user_input_remove == '1':
        fn1()
    elif user_input_remove == '2':
        fn2()
    elif user_input_remove == '3':
        fn3()
    else:
        print('Wrong option!.\n')

In [21]:
def main():
    USER_INPUT = None
    con = connector.connect(user=auth.user,password=auth.password,host=auth.host,database=auth.database)
    my_cursor = con.cursor()
    while True:
        print('Welcome to the Library management system')
        print(
            '''\nPlease choose an option from below menu:\n1. Issue a book\n2. Return a book\n3. View records\n4. Upload records\n5. Remove records\n6. Modify records\n7. Exit\n''')

        try:
            USER_INPUT = int(input())
        except connector.Error as e:
            print('Entered input is not a number please try again.\n')
            print(f'Error : {e}')

        # Issue a book
        if USER_INPUT == 1:
            issue_book()
        # Return a book
        elif USER_INPUT == 2:
            return_book()
        # View records
        elif USER_INPUT == 3:
            select_table(table_books, table_reader, table_book_issue)
        #  Update records
        elif USER_INPUT == 4:
            select_table(add_new_book, add_new_reader)
        # Remove records
        elif USER_INPUT == 5:
            select_table(remove_book, remove_reader)
        # modify records
        elif USER_INPUT == 6:
            select_table(modify_book, modify_reader)
        # exit
        elif USER_INPUT == 7:
            con.close()
            break

## Demo

In [44]:
# 1. Issue a book
main()

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 1


Table : reader


,r_id,r_name
0,1001,Arun Mozhi Devan
1,1002,Rick
2,1003,Beth
3,1004,Sidney


Table : books


...

 3


Table : book_issue


,r_id,b_id
0,1001,4
1,1002,8
2,1003,2
3,1004,1
4,1001,3


Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


In [23]:
# 2. Return a book
main()

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 2


Table : book_issue


,r_id,b_id
0,1001,4
1,1002,8
2,1003,2
3,1004,9
4,1004,1


Enter user ID :  1004
Enter the book id :  9


Book Successfully returned.

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


In [24]:
# 3. View records
main()

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 3


Please select a table :
1. Books
2. Reader
3. Book issue


 1


Table : books


...

 3


Table : book_issue


,r_id,b_id
0,1001,4
1,1002,8
2,1003,2
3,1004,1


Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


In [25]:
# 4. Upload records
main()

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 4


Please select a table :
1. Books
2. Reader


 1


,b_id,b_name,b_author,b_genre,b_availability
0,1,Harry Potter and the Deathly Hallows,J. K. Rowling,Fantasy,Issued
1,2,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,Fantasy,Issued
2,3,Harry Potter and the Philosophers Stone,J. K. Rowling,Fantasy,Available
3,4,Harry Potter and the Goblet of Fire,J. K. Rowling,Fantasy,Available
4,5,Blood of Elves,Andrzej Sapkowski,Fantasy,Issued
5,6,Time of Contempt,Andrzej Sapkowski,Fantasy,Available
6,7,Baptism of Fire,Andrzej Sapkowski,Fantasy,Available
7,8,The Tower of the Swallow,Andrzej Sapkowski,Fantasy,Issued
8,9,The Lady of the Lake,Andrzej Sapkowski,Fantasy,Available


Enter the name of the book :  Thinking, Fast and Slow
Enter the name of the Author :  Daniel Kahneman
Enter the name of the Genre :  Non-fiction


Successfully added Thinking, Fast and Slow user to table reader.

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


In [40]:
# 5. Remove records
main()

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 5


Please select a table :
1. Books
2. Reader


 1


,b_id,b_name,b_author,b_genre,b_availability
0,1,Harry Potter and the Deathly Hallows,J. K. Rowling,Fantasy,Issued
1,2,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,Fantasy,Issued
2,3,Harry Potter and the Philosophers Stone,J. K. Rowling,Fantasy,Available
3,4,Harry Potter and the Goblet of Fire,J. K. Rowling,Fantasy,Available
4,5,Blood of Elves,Andrzej Sapkowski,Fantasy,Issued
5,6,Time of Contempt,Andrzej Sapkowski,Fantasy,Available
6,7,Baptism of Fire,Andrzej Sapkowski,Fantasy,Available
7,8,The Tower of the Swallow,Andrzej Sapkowski,Fantasy,Issued
8,10,"Thinking, Fast and Slow",Daniel Kahneman,Non-fiction,Available


...

 1


,b_id,b_name,b_author,b_genre,b_availability
0,1,Harry Potter and the Deathly Hallows,J. K. Rowling,Fantasy,Issued
1,2,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,Fantasy,Issued
2,3,Harry Potter and the Philosophers Stone,J. K. Rowling,Fantasy,Available
3,4,Harry Potter and the Goblet of Fire,J. K. Rowling,Fantasy,Available
4,5,Blood of Elves,Andrzej Sapkowski,Fantasy,Issued
5,6,Time of Contempt,Andrzej Sapkowski,Fantasy,Available
6,8,The Tower of the Swallow,Andrzej Sapkowski,Fantasy,Issued
7,10,"Thinking, Fast and Slow",Daniel Kahneman,Non-fiction,Available


Enter book ID :  8


Issued book cannot be removed.

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


In [41]:
# 6. Modify records
main()

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 6


Please select a table :
1. Books
2. Reader


 2


,r_id,r_name
0,1001,Arun
1,1002,Rick
2,1003,Beth
3,1004,Sidney


...

 2


Table : reader


,r_id,r_name
0,1001,Arun Mozhi Devan
1,1002,Rick
2,1003,Beth
3,1004,Sidney


Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


In [42]:
# 7. Exit
main()

Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


## Creating a login page

In [22]:
input_username = input('Please enter the user name: ').lower()
if input_username in username:
    pwd_ = hashlib.sha256(getpass('Please enter password: ').encode('utf-8')).hexdigest()
    if pwd_ == username.get(input_username):
        print(f'Welcome {input_username.capitalize()}!')
        main()
    else:
        print('Wrong password')
else:
    print('User not found!')

Please enter the user name:  arun
Please enter password:  ····


Welcome Arun!
Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 3


Please select a table :
1. Books
2. Reader
3. Book issue


 2


Table : reader


,r_id,r_name
0,1001,Arun Mozhi Devan
1,1002,Rick
2,1003,Beth
3,1004,Sidney


Welcome to the Library management system

Please choose an option from below menu:
1. Issue a book
2. Return a book
3. View records
4. Upload records
5. Remove records
6. Modify records
7. Exit



 7


In [23]:
input_username = input('Please enter the user name: ').lower()
if input_username in username:
    pwd_ = hashlib.sha256(getpass('Please enter password: ').encode('utf-8')).hexdigest()
    if pwd_ == username.get(input_username):
        print(f'Welcome {input_username.capitalize()}!')
        main()
    else:
        print('Wrong password')
else:
    print('User not found!')

Please enter the user name:  tony


User not found!


## References

[Getpass](https://docs.python.org/3/library/getpass.html) - Prompt the user for a password without echoing.<br>
[Hashlib](https://docs.python.org/3/library/hashlib.html#module-hashlib) - For generating python secure hash message.<br>
[MySQL Connectors](https://www.mysql.com/products/connector/) - MySQL Connectors provide connectivity to the MySQL server for client programs.<br>
[MySQL Error Exception](https://dev.mysql.com/doc/connector-python/en/connector-python-api-errors-error.html) - Error is internally used by Connector/Python to raise MySQL client.<br>
[Pandas .read_sql_query](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html) - Returns a DataFrame corresponding to the result set of the query string.<br>